In [ ]:
from utils.utils import read_yaml
from utils.model_factory import load_model
import torch

cfg = read_yaml('projects/configs/cfg_LymphPath_17-BRCA.yaml')
model = load_model(cfg)
model.load_state_dict(torch.load('weights/LymphPath.pt'), strict=True)

Using KAN
success to init LymphPath


<All keys matched successfully>

In [ ]:
import pandas as pd
import random
from datasets.ThreeStreamBagDataset import ThreeChannelBagDataset

random.seed(1)
your_csv = pd.read_csv('csv/17-BRCA.csv')

wsi_id = random.choice(list(your_csv['slide_id']))
wsi_data = your_csv[your_csv['slide_id'] == wsi_id]

data_dir1= "./LymphPath_feature/17-BRCA/Gigapath/"
data_dir2= "./LymphPath_feature/17-BRCA/UNI/"
data_dir3= "./LymphPath_feature/17-BRCA/Virchow2/"

dataset = ThreeChannelBagDataset(
    df=wsi_data,
    data_dir1=data_dir1,
    data_dir2=data_dir2,
    data_dir3=data_dir3,
    label_field='label'
)

sample = dataset[0]
model.eval()
model.to('cuda')
feature1 = sample['features1'].to('cuda')
feature2 = sample['features2'].to('cuda')
feature3 = sample['features3'].to('cuda')
y = sample['label'].to('cuda')

res = model(feature1, feature2, feature3, label=y, instance_eval=True)
y_prob = res['y_prob']
print('slide id:', str(wsi_id), 'if LNM:', int(y_prob[:,1] > 0.5))

slide id: patient_013_node_3 if LNM: 1
